In [1]:
import ee
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

ee.Authenticate()
ee.Initialize()

In [2]:
denmark_pts = ee.FeatureCollection("projects/ee-ayang115t/assets/Marker_2018")
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
geometry = ee.Geometry.Polygon([[
    [9.099959167786013, 55.421683620075065],
    [9.099959167786013, 55.337420105686874],
    [9.237288269348513, 55.337420105686874],
    [9.237288269348513, 55.421683620075065]
    ]])

points = denmark_pts.filterBounds(geometry)


In [3]:
df = gpd.GeoDataFrame.from_features(points.getInfo())


In [6]:
unique_codes = df['Afgkode'].unique()
codes = ee.List([ee.String(str(x)) for x in unique_codes])
palette = ee.List([ee.String(x) for x in sns.color_palette("hsv", len(unique_codes)).as_hex()])
color_dict = ee.Dictionary.fromLists(codes, palette)


In [15]:
Map = geemap.Map()
Map.centerObject(points.geometry())

styled = points.map(
  lambda feature:
    feature.set('style', {
      'fillColor': color_dict.get(feature.getNumber('Afgkode').format()),
      'width': 1
    })
  ).style(
    styleProperty='style'
  )
  
Map.add_basemap("HYBRID")
Map.addLayer(styled)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [14]:
Map.to_image('./denmark_map.png')

ImportError: Please install mss package using 'pip install mss'